In [6]:
import keras as ks
from keras.models import Sequential
from keras import backend as K
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, Lambda
from keras.models import Model, Sequential
from keras.initializers import random_normal, zeros
from keras.regularizers import l2
from keras.utils.np_utils import to_categorical
from keras.utils import Sequence
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau
from keras.datasets import cifar10

In [9]:
def VGG16_train(include_top=True, weights='vggface',
          input_tensor=None, input_shape=None,
          pooling=None,weights_path=None,
          classes=8):
    
    model = Sequential()
    
    # Block 1
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(32,32,3)))
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Block 2
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Block 3
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Block 4
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))


    # Block 5
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    #fc layer
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(classes, activation='softmax'))
            
    return model

In [10]:
vgg_model = VGG16_train(classes=10)

vgg_model.compile(optimizer=SGD(lr=0.005, momentum=0.9, decay=0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 8, 8, 256)         295168    
__________

In [15]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [17]:
def preprocess_input(x_temp, data_format=None, version=1):
    if data_format is None:
        data_format = K.image_data_format()
    assert data_format in {'channels_last', 'channels_first'}

    if version == 1:
        if data_format == 'channels_first':
            x_temp = x_temp[:, ::-1, ...]
            x_temp[:, 0, :, :] -= 93.5940
            x_temp[:, 1, :, :] -= 104.7624
            x_temp[:, 2, :, :] -= 129.1863
        else:
            x_temp = x_temp[..., ::-1]
            x_temp[..., 0] -= 93.5940
            x_temp[..., 1] -= 104.7624
            x_temp[..., 2] -= 129.1863

    elif version == 2:
        if data_format == 'channels_first':
            x_temp = x_temp[:, ::-1, ...]
            x_temp[:, 0, :, :] -= 91.4953
            x_temp[:, 1, :, :] -= 103.8827
            x_temp[:, 2, :, :] -= 131.0912
        else:
            x_temp = x_temp[..., ::-1]
            x_temp[..., 0] -= 91.4953
            x_temp[..., 1] -= 103.8827
            x_temp[..., 2] -= 131.0912
    else:
        raise NotImplementedError

    return x_temp

In [18]:
batch_size = 32
num_classes = 10
epochs = 20

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

X_train = preprocess_input(x_train)
X_test = preprocess_input(x_test)

y_train = ks.utils.to_categorical(y_train, num_classes)
y_test = ks.utils.to_categorical(y_test, num_classes)

50000 train samples
10000 test samples


In [20]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, min_lr=0.00001)
batch_size = 64
epochs = 50

In [ ]:
vgg_model.fit(x=X_train, y=y_train, batch_size=batch_size, epochs=epochs,
      validation_data=(X_test, y_test), callbacks=[reduce_lr])

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 42s 832us/step - loss: 1.3475 - acc: 0.5033 - val_loss: 1.1419 - val_acc: 0.5926
Epoch 2/50
50000/50000 [==============================] - 41s 823us/step - loss: 1.0612 - acc: 0.6187 - val_loss: 1.1206 - val_acc: 0.6088
Epoch 3/50
50000/50000 [==============================] - 42s 838us/step - loss: 0.8829 - acc: 0.6879 - val_loss: 0.9035 - val_acc: 0.6893
Epoch 4/50
50000/50000 [==============================] - 42s 843us/step - loss: 0.7347 - acc: 0.7428 - val_loss: 0.8150 - val_acc: 0.7271
Epoch 5/50
50000/50000 [==============================] - 42s 844us/step - loss: 0.6084 - acc: 0.7887 - val_loss: 0.8249 - val_acc: 0.7312
Epoch 6/50
50000/50000 [==============================] - 43s 856us/step - loss: 0.4993 - acc: 0.8271 - val_loss: 0.7722 - val_acc: 0.7510
Epoch 7/50
25344/50000 [==============>...............] - ETA: 20s - loss: 0.3949 - acc: 0.8631